In [61]:
import numpy as np

class SparseMatrix:
    def __init__(self, arr):
        self.coords = np.stack(np.nonzero(arr), 1)
        self.values = arr[np.nonzero(arr)]
        self.shape = arr.shape

    def scalarProduct(self, scalar):
        new_obj = self.__new__(type(self))
        new_obj.values *= scalar

        return new_obj
    
    def equals(self, lhs, rhs):
        return all(lhs == rhs)
    
    def less(self, lhs, rhs):
        if lhs[0] < rhs[0]:
            return True
        return lhs[1] < lhs[2]
        
    def sum(self, other):

        if not all(self.shape == other.shape):
            raise Exception('sizes mismatch')

        res = SparseMatrix([])

        len_l = len(self.coords)
        len_r = len(other.coords)

        i_l = i_r = 0
        while i_l < len_l and i_r < len_r:
            if self.equals(self.coords[i_l], other.coords[i_r]):
                res.values.append(self.values[i_l] + self.values[i_r])
                res.coords.append(self.coords[i_l])
                i_l += 1
                i_r += 1
            elif self.less(self.coords[i_l], other.coords[i_r]):
                res.values.append(self.values[i_l])
                res.coords.append(self.coords[i_l])
                i_l += 1
            else:
                res.values.append(self.values[i_r])
                res.coords.append(self.coords[i_r])
                i_r += 1

        while i_l < len_l:
            res.values.append(self.values[i_l])
            res.coords.append(self.coords[i_l])
            i_l += 1
        
        while i_r < len_r:
            res.values.append(self.values[i_r])
            res.coords.append(self.coords[i_r])
            i_r += 1      

        res.shape = self.shape
        
        return res
    
    def matVec(self, vector):

        if self.shape[1] != vector.shape[0]:
            raise Exception('sizes mismatch')
        
        if vector.shape[1] != 1:
            raise Exception(f'{vector} is not vector')
        
        res = np.zeros((self.shape[0], 1))
        
        for i in range(len(self.values)):
            idx_vec = self.coords[i][1]
            idx_res = self.coords[i][0]
            res[idx_res][0] += self.values[i] * vector[idx_vec][0]

        return res


In [62]:
def equals(lhs, rhs):
    return all(lhs == rhs)

def less(lhs, rhs):
    if lhs[0] < rhs[0]:
        return True
    return lhs[1] < lhs[2]

In [63]:
import numpy as anp

input_arr = np.stack(np.nonzero(np.eye(3)), 1)

In [64]:
arr = np.diag(np.arange(1, 4))
arr

array([[1, 0, 0],
       [0, 2, 0],
       [0, 0, 3]])

In [66]:
obj = SparseMatrix(arr)
obj.values, obj.coords

(array([1, 2, 3]),
 array([[0, 0],
        [1, 1],
        [2, 2]]))

In [69]:
%%time

obj.matVec(np.ones((3, 1)) * 2)

CPU times: user 282 µs, sys: 2.33 ms, total: 2.61 ms
Wall time: 7.65 ms


array([[2.],
       [4.],
       [6.]])

In [70]:
%%time

np.diag([1, 2, 3]) @ np.ones(3)

CPU times: user 177 µs, sys: 526 µs, total: 703 µs
Wall time: 2.23 ms


array([1., 2., 3.])